In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/COMPETENCIAS /NASA - COVID - MIT_critical_data/data

/content/drive/MyDrive/COMPETENCIAS /NASA - COVID - MIT_critical_data/data


In [3]:
!pip install geopandas
!pip install geojson

     |████████████████████████████████| 1.0 MB 10.2 MB/s 
     |████████████████████████████████| 15.4 MB 44 kB/s 
     |████████████████████████████████| 6.3 MB 35.9 MB/s 


In [4]:
import json
import geopandas as gpd
import pandas as pd
import geojson

In [7]:
departments = "departments_and_risk.geosjon"
# Opening JSON file
f = open(departments,)
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
data

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'AREA_OFICI': 1605,
    'FID': 1,
    'ID_ESPACIA': '11',
    'NOMBRE_DPT': 'bogota',
    'risk': 2.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-74.0696436581778, 4.82983260498634],
      [-74.0694575540154, 4.82981708637489],
      [-74.0692889209605, 4.8298270366282],
      [-74.0689763652432, 4.82989925448052],
      [-74.0686932036631, 4.83001785170221],
      [-74.0685371149162, 4.83010269401866],
      [-74.0682822268923, 4.83026902601664],
      [-74.0680977428917, 4.83040976981371],
      [-74.0678163215647, 4.83065473095337],
      [-74.0675030694803, 4.83096463015788],
      [-74.0662745005007, 4.83234929089392],
      [-74.0658393582815, 4.83280905362554],
      [-74.0655350959848, 4.83309977904961],
      [-74.0652741481101, 4.83332617879261],
      [-74.0648585304576, 4.83364108691027],
      [-74.0646311234869, 4.83378888859988],
      [-74.0643353935925, 4.833

In [8]:
df_places = gpd.read_file(departments)
df_places.NOMBRE_DPT[df_places.NOMBRE_DPT == 'archipielagodesanandres,providenciaysantacatalina'] = 'sanandres'
df_places.NOMBRE_DPT = df_places.NOMBRE_DPT.str.strip()
df_places.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


id  AREA_OFICI  ...  risk                                           geometry
0  0        1605  ...   2.0  POLYGON ((-74.06964 4.82983, -74.06946 4.82982...
1  1       26383  ...   1.0  POLYGON ((-75.25600 10.79971, -75.25547 10.799...
2  2       23189  ...   0.0  POLYGON ((-72.15949 7.04211, -72.15852 7.04148...
3  3        7888  ...   2.0  POLYGON ((-74.67036 5.77004, -74.66925 5.76905...
4  4       88965  ...   0.0  POLYGON ((-74.78494 2.92441, -74.78492 2.89536...

[5 rows x 7 columns]

In [9]:
# To unite
spatial_data_path = "spatial_data_merge_by_date.csv"

spatial_data = pd.read_csv(spatial_data_path)

mutations = pd.read_csv('Covid19_mu_preprocessed.csv')

spatial_data.drop(columns=['Unnamed: 0'], inplace=True)
mutations.drop(columns=['Unnamed: 0'], inplace=True)

spatial_data = spatial_data.groupby(["Date"]).mean().reset_index()


x = [ ]
for i in spatial_data["Date"]:
  x.append(i + ' 00:00:00')
spatial_data["Date"] = x
spatial_data.head(5)

mutations = mutations.merge(spatial_data, how='inner', on=['Date'])
mutations = mutations.reset_index()

In [10]:
mutations.columns

Index(['index', 'Date', 'amazonas', 'antioquia', 'arauca', 'atlantico',
       'bogota', 'bolivar', 'boyaca', 'caldas', 'caqueta', 'casanare', 'cesar',
       'choco', 'cordoba', 'cundinamarca', 'guaviare ', 'huila', 'laguajira',
       'magdalena', 'meta', 'narino', 'nortedesantander', 'quindio',
       'risaralda', 'sanandres', 'santander', 'sucre', 'tolima',
       'valledelcauca', 'vaupes', 'vichada', 'T2M', 'T2MDEW', 'TS',
       'PRECTOTCORR', 'PS'],
      dtype='object')

In [12]:
#mutations.drop(columns=['Unnamed: 0'], inplace=True)
df_copy = mutations.copy() 
df_copy =df_copy[['index', 'Date', 'amazonas', 'antioquia', 'arauca', 'atlantico',
       'bogota', 'bolivar', 'boyaca', 'caldas', 'caqueta', 'casanare', 'cesar',
       'choco', 'cordoba', 'cundinamarca', 'guaviare ', 'huila', 'laguajira',
       'magdalena', 'meta', 'narino', 'nortedesantander', 'quindio',
       'risaralda', 'sanandres', 'santander', 'sucre', 'tolima',
       'valledelcauca', 'vaupes', 'vichada']]

df_copy.index = df_copy.Date
df_copy = df_copy.T
df_copy = df_copy.iloc[1:].reset_index()
df_copy.rename(columns={'index':'NOMBRE_DPT'}, inplace=True)
df_copy.NOMBRE_DPT = df_copy.NOMBRE_DPT.str.strip()
df_copy = df_copy[1:]


df_copy['T2M']    = mutations["T2M"]
df_copy['T2MDEW'] = mutations["T2MDEW"]
df_copy['TS']     = mutations["TS"]
df_copy['PRECTOTCORR'] = mutations["PRECTOTCORR"]
df_copy['PS']     = mutations["PS"]
df_copy

Date        NOMBRE_DPT 2021-04-03 00:00:00  ... PRECTOTCORR         PS
1             amazonas                   0  ...    0.550494  87.587037
2            antioquia                   0  ...    0.281111  87.589630
3               arauca                   0  ...    8.832840  87.657284
4            atlantico                   4  ...    1.041852  87.698765
5               bogota                   0  ...    5.028272  87.698889
6              bolivar                   0  ...    3.426667  87.682099
7               boyaca                   0  ...    1.134938  87.701975
8               caldas                   0  ...    6.199753  87.645309
9              caqueta                   0  ...    6.986049  87.670494
10            casanare                   0  ...   16.771111  87.732222
11               cesar                   0  ...   21.103827  87.764074
12               choco                   0  ...   16.125802  87.836420
13             cordoba                   0  ...    3.030741  87.807778
14        cundinamarca                   0  ...    4.318025  87.794568
15            guaviare                   0  ...    1.622222  87.678519
16               huila                   0  ...    0.351605  87.706420
17           laguajira                   0  ...    0.593086  87.715802
18           magdalena                   0  ...    3.285556  87.683580
19                meta                   0  ...    3.440123  87.645926
20              narino                   0  ...    2.724938  87.674938
21    nortedesantander                   0  ...    3.054321  87.765185
22             quindio                   0  ...    3.589877  87.838395
23           risaralda                   0  ...    1.682099  87.919753
24           sanandres                   0  ...    0.581728  87.943457
25           santander                   0  ...    0.792840  87.915802
26               sucre                   0  ...    0.272593  87.931358
27              tolima                   0  ...    2.125309  87.844198
28       valledelcauca                   0  ...    0.683086  87.850494
29              vaupes                   0  ...    0.932963  87.843951
30             vichada                   0  ...    0.326173  87.716173

[30 rows x 102 columns]

In [13]:
df_copy = df_copy.merge(df_places, how='inner', on=['NOMBRE_DPT'])
mutations = df_copy.reset_index()
mutations

index  ...                                           geometry
0       0  ...  POLYGON ((-71.28883 0.06675, -71.28846 0.06669...
1       1  ...  POLYGON ((-76.40834 8.74648, -76.40755 8.74623...
2       2  ...  POLYGON ((-70.67987 7.09254, -70.67905 7.09115...
3       3  ...  POLYGON ((-74.83155 11.05222, -74.82395 11.046...
4       4  ...  POLYGON ((-74.06964 4.82983, -74.06946 4.82982...
5       5  ...  POLYGON ((-75.25600 10.79971, -75.25547 10.799...
6       6  ...  POLYGON ((-72.15949 7.04211, -72.15852 7.04148...
7       7  ...  POLYGON ((-74.67036 5.77004, -74.66925 5.76905...
8       8  ...  POLYGON ((-74.78494 2.92441, -74.78492 2.89536...
9       9  ...  POLYGON ((-70.36222 6.30055, -70.36084 6.29916...
10     10  ...  POLYGON ((-73.39171 10.86598, -73.39168 10.865...
11     11  ...  POLYGON ((-77.36540 8.67342, -77.36437 8.67305...
12     12  ...  POLYGON ((-75.87806 9.42309, -75.87627 9.42305...
13     13  ...  POLYGON ((-74.31495 5.80046, -74.31410 5.80039...
14     14  ...  POLYGON ((-71.28368 2.86782, -71.28071 2.86767...
15     15  ...  POLYGON ((-74.51814 3.82550, -74.51267 3.82439...
16     16  ...  POLYGON ((-71.66600 12.45679, -71.66572 12.456...
17     17  ...  POLYGON ((-74.04537 11.34946, -74.04339 11.349...
18     18  ...  POLYGON ((-71.07957 4.92497, -71.07850 4.92472...
19     19  ...  POLYGON ((-78.14140 2.67691, -78.14140 2.67202...
20     20  ...  POLYGON ((-73.00339 9.28749, -73.00328 9.28744...
21     21  ...  POLYGON ((-75.66522 4.70053, -75.66044 4.69892...
22     22  ...  POLYGON ((-76.00835 5.56453, -76.00825 5.56368...
23     23  ...  MULTIPOLYGON (((-81.70714 12.59750, -81.70711 ...
24     24  ...  POLYGON ((-73.72309 7.96963, -73.72066 7.96789...
25     25  ...  POLYGON ((-75.51494 10.02522, -75.51127 10.025...
26     26  ...  POLYGON ((-74.82061 5.31515, -74.81987 5.31468...
27     27  ...  POLYGON ((-76.08427 5.03608, -76.07898 5.03500...
28     28  ...  POLYGON ((-70.11545 2.06455, -70.11569 2.06376...
29     29  ...  POLYGON ((-67.75989 6.29114, -67.75962 6.29114...

[30 rows x 109 columns]

In [14]:
mutations.to_csv('/Covid19_mu_preprocessed_final.csv')

### Population

In [16]:
population = pd.read_csv('Census_data.csv')[['total_population', 'Dpto']]
population.rename(columns={'Dpto':'NOMBRE_DPT'}, inplace=True)
population

total_population        NOMBRE_DPT
0           359602.0           caqueta
1          1243503.0             cauca
2           283197.0          putumayo
3          3789874.0     valledelcauca
4            44431.0           guainia
5            76642.0           vichada
6           379892.0          casanare
7            66056.0          amazonas
8            37690.0            vaupes
9            73081.0          guaviare
10          923472.0            caldas
11          509640.0           quindio
12          839597.0         risaralda
13         5974788.0         antioquia
14          457412.0             choco
15         1335521.0            narino
16         1555596.0           cordoba
17         1909460.0           bolivar
18         1098577.0             cesar
19          825364.0         laguajira
20         1263788.0         magdalena
21          864036.0             sucre
22           48299.0         sanandres
23          239503.0            arauca
24         1135698.0            boyaca
25         2792877.0      cundinamarca
26         1346806.0  nortedesantander
27         7181469.0            bogota
28          919129.0              meta
29         1009548.0             huila
30         2008841.0         santander
31         1228763.0            tolima
32         2342265.0         atlantico

In [17]:
mutations = mutations.merge(population, on=['NOMBRE_DPT'],how='inner')

In [ ]:
#mutations["geometry"]

In [18]:
poly = {
        'type': 'Polygon',
        'coordinates': mutations["geometry"]
}

In [19]:
data = {
    'type': 'features',
    'id':mutations['index'],
    'properies':{        
        'name':mutations['NOMBRE_DPT'],
        'risk':mutations['risk']
    }
}

In [20]:
df = {
    'type': 'FeatureCollection',
    'features': data,
    'geometry': poly 

}

In [21]:
gdf = gpd.GeoDataFrame(mutations)
gdf

index  ... total_population
0       0  ...          66056.0
1       1  ...        5974788.0
2       2  ...         239503.0
3       3  ...        2342265.0
4       4  ...        7181469.0
5       5  ...        1909460.0
6       6  ...        1135698.0
7       7  ...         923472.0
8       8  ...         359602.0
9       9  ...         379892.0
10     10  ...        1098577.0
11     11  ...         457412.0
12     12  ...        1555596.0
13     13  ...        2792877.0
14     14  ...          73081.0
15     15  ...        1009548.0
16     16  ...         825364.0
17     17  ...        1263788.0
18     18  ...         919129.0
19     19  ...        1335521.0
20     20  ...        1346806.0
21     21  ...         509640.0
22     22  ...         839597.0
23     23  ...          48299.0
24     24  ...        2008841.0
25     25  ...         864036.0
26     26  ...        1228763.0
27     27  ...        3789874.0
28     28  ...          37690.0
29     29  ...          76642.0

[30 rows x 110 columns]

In [22]:
with open('data_mu.geojson','w') as f:
    geojson.dump(gdf, f)

In [23]:
departments = "data_mu.geojson"
# Opening JSON file
f = open(departments,)
 
# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
data

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'2021-01-04 00:00:00': 0.0,
    '2021-01-05 00:00:00': 0.0,
    '2021-01-06 00:00:00': 0.0,
    '2021-01-07 00:00:00': 0.0,
    '2021-01-08 00:00:00': 0.0,
    '2021-02-03 00:00:00': 0.0,
    '2021-02-04 00:00:00': 0.0,
    '2021-02-05 00:00:00': 0.0,
    '2021-02-06 00:00:00': 0.0,
    '2021-02-07 00:00:00': 0.0,
    '2021-02-08 00:00:00': 0.0,
    '2021-02-21 00:00:00': 0.0,
    '2021-02-26 00:00:00': 0.0,
    '2021-02-27 00:00:00': 0.0,
    '2021-03-03 00:00:00': 0.0,
    '2021-03-04 00:00:00': 0.0,
    '2021-03-05 00:00:00': 0.0,
    '2021-03-06 00:00:00': 0.0,
    '2021-03-07 00:00:00': 0.0,
    '2021-03-08 00:00:00': 0.0,
    '2021-03-14 00:00:00': 0.0,
    '2021-03-15 00:00:00': 0.0,
    '2021-03-17 00:00:00': 0.0,
    '2021-03-18 00:00:00': 0.0,
    '2021-03-19 00:00:00': 0.0,
    '2021-03-21 00:00:00': 0.0,
    '2021-03-23 00:00:00': 0.0,
    '2021-03-24 00:00:00': 0.0,
    '2021-03